In [1]:
import uuid

from hana_ml.algorithms.pal.auto_ml import AutomaticClassification
from hana_ml.dataframe import ConnectionContext
from hana_ml.model_storage import ModelStorage
from hana_ml.visualizers.automl_progress import PipelineProgressStatusMonitor
from hana_ml.visualizers.automl_report import BestPipelineReport
from hana_ml.visualizers.unified_report import UnifiedReport

## Check the setup and connect to the database

In [ ]:
%run '010-check_setup.ipynb'

# Tables from SAP HANA

In [3]:
column_id="PASSENGERID"
column_label="SURVIVED"
features_subset=[
            "NAMEFIRSTLETTER",
            "FAREPERPERSON",
            "SIBSP",
            "PARCH",
            "SEX",
            "PCLASS",
            "EMBARKED",
            "TITLE",
            "AGE",
            "PERSONSPERTICKET"
]

In [4]:
hdf_titanic_train = myconn.table("DATA_LABELED_FE").select(
            [column_id]+features_subset+[column_label]
        ).cast(column_label, 'NVARCHAR(1)')

# Instantiate AutoML `auto_c` object

Parametrization of `auto_c` instance: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2024_1_QRC/en-US/pal/topics/genetic_optimization_automl.html#control-parameters

In [5]:
# AutomaticClassification init 
progress_id = "automl_{}".format(uuid.uuid1())
auto_c = AutomaticClassification(
    progress_indicator_id=progress_id
)

In [6]:
# enable_workload_class
auto_c.enable_workload_class(workload_class_name="PAL_AUTOML_WORKLOAD")

Display the current configuration

In [ ]:
auto_c.display_config_dict()

## Get an overview of pipeline operators and parameters

In [8]:
from hana_ml.algorithms.pal import auto_ml

In [ ]:
auto_ml.get_pipeline_info(myconn).collect()

In [ ]:
from IPython.display import HTML
HTML(auto_ml.get_pipeline_info(myconn).filter("NAME='RDT_Classifier'").collect().to_html())

In [ ]:
eval(auto_ml.get_pipeline_info(myconn).filter("NAME='RDT_Classifier'").collect()["PARAMETERS"][0])

## Experiment with [pipeline setting](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_3_QRC/en-US/pal/algorithms/hana_ml.algorithms.pal.auto_ml.AutomaticClassification.html#hana_ml.algorithms.pal.auto_ml.AutomaticClassification.delete_config_dict)

In [ ]:
auto_c.delete_config_dict(category="Resampler")
auto_c.delete_config_dict(category="Transformer")

In [ ]:
auto_c.delete_config_dict(operator_name='MLP_Classifier')
auto_c.delete_config_dict(operator_name='M_LOGR_Classifier')
auto_c.delete_config_dict(operator_name='NB_Classifier')
auto_c.delete_config_dict(operator_name='HGBT_Classifier')
auto_c.delete_config_dict(operator_name='SVM_Classifier')

In [ ]:
auto_c.update_config_dict("RDT_Classifier", "TREES_NUM", [11, 49, 99])
auto_c.update_config_dict("RDT_Classifier", "MAX_DEPTH", {'range': [3, 2, 8]})
auto_c.update_config_dict("RDT_Classifier", "NODE_SIZE", [1, 100])
auto_c.delete_config_dict(operator_name='RDT_Classifier', param_name='SAMPLE_FRACTION')

In [ ]:
auto_c.display_config_dict()

## Start Progress Status Monitor

In [ ]:
# invoke a PipelineProgressStatusMonitor
progress_status_monitor = PipelineProgressStatusMonitor(
    # need to create a connection context different from the one of AutoML
    connection_context=myconn, 
    automatic_obj=auto_c)

In [ ]:
progress_status_monitor.start()

# training
try:
    auto_c.fit(data=hdf_titanic_train.set_index('PassengerId'))
except Exception as e:
    raise e

## Get a report for the Best Pipeline

In [ ]:
BestPipelineReport(auto_c).generate_notebook_iframe()

In [ ]:
# To generate an HTML page for the same:
# BestPipelineReport(auto_c).generate_html('auto-ml')

In [ ]:
auto_c.model_[0].head(5).collect()

In [ ]:
auto_c.model_[1].collect()

## Save the model in SAP HANA db

In [ ]:
ms = ModelStorage(myconn)
auto_c.name = 'CodeJam-Titanic-AutoML'
auto_c.version = 1
ms.save_model(model=auto_c, if_exists='replace')

In [ ]:
ms.list_models()

In [ ]:
null = None
false = False
true = True
eval(ms.list_models(name=auto_c.name, version=auto_c.version).at[0, 'JSON'])

# Call prediction

In [ ]:
hdf_titanic_test=myconn.table('DATA_TO_PREDICT_FE').select(
            [column_id]+features_subset
        )

In [ ]:
hdf_res = auto_c.predict(hdf_titanic_test.set_index('PassengerId'))

In [ ]:
hdf_res.head(4).collect()

# Compare to the ground truth

In [ ]:
hdf_titanic_complete=myconn.table('DATA_COMPLETE', schema='TITANIC')

In [ ]:
hdf_res_ext=hdf_res.set_index('ID').join(myconn.table("DATA_TO_PREDICT_FE").set_index('PassengerId'))

In [ ]:
hdf_res_ext.head(3).collect()

In [ ]:
hdf_res_incl_groundtruth=(hdf_res_ext.set_index(['Name', 'Ticket']).join(hdf_titanic_complete.set_index(['name', 'ticket']))
                 .select('ID', 'Name', 'Ticket', 'SCORES','survived',('1-ABS(SCORES-"survived")', 'IS_CORRECT'))
                 .cast('SCORES', 'INT')

)

In [ ]:
hdf_res_incl_groundtruth.head(3).collect()

## Using `metrics`

In [ ]:
import hana_ml.algorithms.pal.metrics as pal_metrics

In [ ]:
pal_metrics.accuracy_score(data=hdf_res_incl_groundtruth, label_true='survived', label_pred='SCORES')

In [ ]:
hdf_cm, hdf_cr = pal_metrics.confusion_matrix(data=hdf_res_incl_groundtruth, key='ID', label_true='survived', label_pred='SCORES')

In [ ]:
hdf_cm.collect()

In [ ]:
hdf_cr.collect()

In [ ]:
from hana_ml.visualizers.metrics import MetricsVisualizer
MetricsVisualizer(enable_plotly=False).plot_confusion_matrix(hdf_cm, normalize=False);

🤓 **Let's discuss**:
- Comparison of the last two models you trained